# Facial Expression Classification (on color image)

In [26]:
# !echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
# !apt -qq update
# !apt -qq install gcsfuse

In [27]:
# !mkdir colab_direcoty
# !gcsfuse --implicit-dirs fm_grayscale colab_direcoty

In [28]:
# !ls colab_direcoty

In [29]:
import pandas as pd

data_train = pd.read_csv("https://storage.googleapis.com/fm_grayscale/color_train.csv")
data_valid = pd.read_csv("https://storage.googleapis.com/fm_grayscale/color_valid.csv")
data_test = pd.read_csv("https://storage.googleapis.com/fm_grayscale/color_test.csv")
data_train

,img_name,img_array,label
0,Angry.9677.jpg,[[ 33 35 35 ... 40 37 34]\n [ 38 31 30 ...,angry
1,Angry.26462.jpg,[[103 103 95 ... 169 189 177]\n [ 87 81 68 ...,angry
2,19187Exp0frightened_programmer_292.jpg,[[180 178 176 ... 174 172 170]\n [179 178 176 ...,angry
3,5035Exp0awe_teacher_414.jpg,[[ 77 71 69 ... 48 104 217]\n [ 69 66 66 ...,angry
4,18407Exp0frightened_by_the_movie_woman_2.jpg,[[ 29 31 72 ... 255 253 246]\n [ 27 64 58 ...,angry
...,...,...,...
92963,Surprise.20439.jpg,[[250 204 118 ... 244 255 246]\n [245 138 58 ...,surprise
92964,Surprise.21727.jpg,[[ 81 49 0 ... 170 152 150]\n [ 51 0 62 ...,surprise
92965,46523Exp5amazed_boy_143.jpg,[[250 254 247 ... 105 118 104]\n [254 249 252 ...,surprise
92966,75069Exp5afraid_father_738.jpg,[[179 145 69 ... 17 15 19]\n [134 75 29 ...,surprise


In [30]:
data_train

,img_name,img_array,label
0,Angry.9677.jpg,[[ 33 35 35 ... 40 37 34]\n [ 38 31 30 ...,angry
1,Angry.26462.jpg,[[103 103 95 ... 169 189 177]\n [ 87 81 68 ...,angry
2,19187Exp0frightened_programmer_292.jpg,[[180 178 176 ... 174 172 170]\n [179 178 176 ...,angry
3,5035Exp0awe_teacher_414.jpg,[[ 77 71 69 ... 48 104 217]\n [ 69 66 66 ...,angry
4,18407Exp0frightened_by_the_movie_woman_2.jpg,[[ 29 31 72 ... 255 253 246]\n [ 27 64 58 ...,angry
...,...,...,...
92963,Surprise.20439.jpg,[[250 204 118 ... 244 255 246]\n [245 138 58 ...,surprise
92964,Surprise.21727.jpg,[[ 81 49 0 ... 170 152 150]\n [ 51 0 62 ...,surprise
92965,46523Exp5amazed_boy_143.jpg,[[250 254 247 ... 105 118 104]\n [254 249 252 ...,surprise
92966,75069Exp5afraid_father_738.jpg,[[179 145 69 ... 17 15 19]\n [134 75 29 ...,surprise


In [31]:
def change_train_dir(img):
  return "https://storage.googleapis.com/fm_grayscale/train7000/" + img
  # return "train/" + img

def change_valid_dir(img):
  # return "colab_direcoty/valid/" + img
  return "https://storage.googleapis.com/fm_grayscale/valid/" + img

def change_test_dir(img):
  # return "colab_direcoty/test/" + img
  return "https://storage.googleapis.com/fm_grayscale/test700/" + img

data_train["img_name"] = data_train["img_name"].apply(change_train_dir)
data_valid["img_name"] = data_valid["img_name"].apply(change_valid_dir)
data_test["img_name"] = data_test["img_name"].apply(change_test_dir)


In [32]:
data_train["img_name"][0]

'https://storage.googleapis.com/fm_grayscale/train7000/10012Exp0fighting_people_350.jpg'

In [33]:
! pip install tensorflow
! pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils

image_processor = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [35]:
train_gen = image_processor.flow_from_dataframe(
 data_train,
    batch_size = 64,
    x_col='img_name',
    y_col='label',
    target_size = (48, 48),
    shuffle=False,
    color_mode = "rgb",
    class_mode="categorical",
)

valid_gen = image_processor.flow_from_dataframe(
 data_valid,
    batch_size = 64,
    x_col='img_name',
    y_col='label',
    target_size = (48, 48),
    shuffle=True,
    color_mode = "rgb",
    class_mode="categorical",
)

test_gen = image_processor.flow_from_dataframe(
 data_test,
    batch_size = 64,
    x_col='img_name',
    y_col='label',
    target_size = (48, 48),
    shuffle=True,
    color_mode = "rgb",
    class_mode="categorical",
)

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 92968 invalid image filename(s) in x_col="img_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 17356 invalid image filename(s) in x_col="img_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [36]:
train_gen.image_shape

(48, 48, 3)

In [37]:
batch_size = 512

cnn_model= Sequential([
    Conv2D(filters=36, kernel_size=7, activation='relu', input_shape=(48, 48, 3)),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=54, kernel_size=5, activation='relu', input_shape=(48, 48, 3)),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(2024, activation='relu'),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    #20 is the number of outputs
    Dense(7, activation='softmax')  
])

cnn_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=0.0001),
    metrics=['accuracy']
)

# cnn_model.fit_generator( train_gen,
#     steps_per_epoch = train_gen.samples // batch_size,
#     validation_data = valid_gen, 
#     validation_steps = valid_gen.samples // batch_size,
#     epochs = 100,
#     )

history = cnn_model.fit(
    train_gen,
    batch_size=512,
    epochs=100, 
    verbose=2,
    validation_data=valid_gen,
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


ValueError: ignored

In [ ]:
cnn_model.evaluate(test_gen)

In [ ]:
import seaborn as sns
from matplotlib import pyplot

sns.set()
fig = pyplot.figure(0, (12, 4))

ax = pyplot.subplot(1, 2, 1)
sns.lineplot(history.epoch, history.history['accuracy'], label='train')
sns.lineplot(history.epoch, history.history['val_accuracy'], label='valid')
pyplot.title('Accuracy')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.lineplot(history.epoch, history.history['loss'], label='train')
sns.lineplot(history.epoch, history.history['val_loss'], label='valid')
pyplot.title('Loss')
pyplot.tight_layout()

pyplot.savefig('epoch_history_dcnn_color.png')
pyplot.show()

In [ ]:
df_accu = pd.DataFrame({'train': history.history['accuracy'], 'valid': history.history['val_accuracy']})
df_loss = pd.DataFrame({'train': history.history['loss'], 'valid': history.history['val_loss']})

fig = pyplot.figure(0, (14, 4))
ax = pyplot.subplot(1, 2, 1)
sns.violinplot(x="variable", y="value", data=pd.melt(df_accu), showfliers=False)
pyplot.title('Accuracy')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.violinplot(x="variable", y="value", data=pd.melt(df_loss), showfliers=False)
pyplot.title('Loss')
pyplot.tight_layout()

pyplot.savefig('performance_dist_color.png')
pyplot.show()

In [ ]:
!pip install scikit-plot

In [ ]:
import scikitplot
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay, classification_report
import numpy as np

def plot_cm(X_test, y_test):
  y_predicted = cnn_model.predict_generator(X_test).argmax(axis=1)
  cm_label = ["anger", "disgust", "fear", "happiness", "sadness", "surprise", "neutral"]
  scikitplot.metrics.plot_confusion_matrix(y_test, y_predicted, figsize=(7,7))
  pyplot.savefig("confusion_matrix_color.png")

  print(f'total wrong validation predictions: {np.sum(y_predicted != y_test)}\n\n')
  print(classification_report(y_predicted, y_test))

In [ ]:
plot_cm(test_gen, test_gen.labels)